<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("OECD_HPI.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,cou,FREQUENCY,ind,indexed_at,provider_code,series_code,series_name,series_num
0,1970,1970-01-01,2.552380,annual,HOUSE_PRICES,Analytical house prices indicators,AUS,A,HPI,2024-10-31T09:51:20.215Z,OECD,A.AUS.HPI,Annual – Australia – Nominal house price indic...,Master
1,1971,1971-01-01,2.851352,annual,HOUSE_PRICES,Analytical house prices indicators,AUS,A,HPI,2024-10-31T09:51:20.215Z,OECD,A.AUS.HPI,Annual – Australia – Nominal house price indic...,Master
2,1972,1972-01-01,3.183667,annual,HOUSE_PRICES,Analytical house prices indicators,AUS,A,HPI,2024-10-31T09:51:20.215Z,OECD,A.AUS.HPI,Annual – Australia – Nominal house price indic...,Master
3,1973,1973-01-01,3.837183,annual,HOUSE_PRICES,Analytical house prices indicators,AUS,A,HPI,2024-10-31T09:51:20.215Z,OECD,A.AUS.HPI,Annual – Australia – Nominal house price indic...,Master
4,1974,1974-01-01,4.690199,annual,HOUSE_PRICES,Analytical house prices indicators,AUS,A,HPI,2024-10-31T09:51:20.215Z,OECD,A.AUS.HPI,Annual – Australia – Nominal house price indic...,Master


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [2]:
python_df = pd.read_pickle("OECD_HPI.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,FREQUENCY,COU,IND,Frequency,Country,Indicator
0,annual,OECD,HOUSE_PRICES,Analytical house prices indicators,A.AUS.HPI,Annual – Australia – Nominal house price indic...,1970,1970-01-01,2.552380,2.552380,A,AUS,HPI,Annual,Australia,"Nominal house price indices, s.a."
1,annual,OECD,HOUSE_PRICES,Analytical house prices indicators,A.AUS.HPI,Annual – Australia – Nominal house price indic...,1971,1971-01-01,2.851352,2.851352,A,AUS,HPI,Annual,Australia,"Nominal house price indices, s.a."
2,annual,OECD,HOUSE_PRICES,Analytical house prices indicators,A.AUS.HPI,Annual – Australia – Nominal house price indic...,1972,1972-01-01,3.183667,3.183667,A,AUS,HPI,Annual,Australia,"Nominal house price indices, s.a."
3,annual,OECD,HOUSE_PRICES,Analytical house prices indicators,A.AUS.HPI,Annual – Australia – Nominal house price indic...,1973,1973-01-01,3.837183,3.837183,A,AUS,HPI,Annual,Australia,"Nominal house price indices, s.a."
4,annual,OECD,HOUSE_PRICES,Analytical house prices indicators,A.AUS.HPI,Annual – Australia – Nominal house price indic...,1974,1974-01-01,4.690199,4.690199,A,AUS,HPI,Annual,Australia,"Nominal house price indices, s.a."


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [3]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [4]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce').astype('float64')
python_df['value'] = python_df['value'].astype('float64')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [5]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [6]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True